# 필요한 모듈들을 임포트

In [2]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split
import cv2
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import save_model,load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input ,GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from functools import partial

# 이미지 배열 생성 및 파일 저장 

In [2]:
caltech_dir = "./HandGesture\images" #학습데이터
categories = ["O", "V", "paper", "rock","side"] #카테고리 나누기
nb_classes = len(categories)#카테고리 길이 결정
image_w = 64
image_h = 64
pixels = image_h * image_w * 3 #픽셀은 64*64에 3층
X = []
y = []
#카테고리 수만큼 전처리 원핫 인코딩
for idx, cat in enumerate(categories): 
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    image_dir = caltech_dir + "/" + cat #cat의 폴더 열기
    files = glob.glob(image_dir+"/*.jpg") #cat에 있는 jpg로 끝나는 파일 읽어오기
    print(cat, " 파일 길이 : ", len(files))#전체 파일 수 i 파일 수 f는 파일 이름
    for i, f in enumerate(files):#파일 수만큼 반복
        img = cv2.imread(f)#이미지 열기 cv2로 열기
        #손 색상 변경
        ycrb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb) #cv2의 색상을 RGB가 아닌 YCrCb로 변경
        img = cv2.inRange(ycrb,np.array([0,133,77]),np.array([255,173,127]))  #해당 색(사람 피부색)에 관련된 색을 구분
        
        img = Image.fromarray(img) #cv2의 이미지를 PIL형식의 이미지로 변경
        img = img.convert("RGB")#흑백
        img = img.resize((image_w, image_h))#사이즈 변경
        # img.show()
        data = np.asarray(img)#np.array로 변경
        X.append(data) #x에 데이터 
        y.append(label) #y에 출력 설정

X = np.array(X) # 데이터 전부 모아서 배열 성정
y = np.array(y)

#학습집합 나누어 주기

#X_train y_train 최종학습 집합
#X_train_f y_train_f 학습 집합
#x_test y_test 테스트 집합
#X_vailed y_vailed 검증 집합
X_train, X_test, y_train, y_test = train_test_split(X, y) 
X_train_f, X_vailed, y_train_f, y_vailed = train_test_split(X_train,y_train)
xy = (X_train, X_test, y_train, y_test,X_train_f, X_vailed, y_train_f, y_vailed)


np.savez("./HandGesture/multi_image_data.npy", X_train=X_train, X_test=X_test,y_train=y_train,y_test=y_test,X_train_f=X_train_f,X_vailed=X_vailed, y_train_f=y_train_f,y_vailed=y_vailed )
print("ok", len(y))

O  파일 길이 :  421
V  파일 길이 :  465
paper  파일 길이 :  435
rock  파일 길이 :  450
side  파일 길이 :  453
ok 2224


# 저장했던 데이터 불러오기 및 이미지 증식 

In [4]:
# 데이터 증식 옵션
train_datagen = ImageDataGenerator(rescale = 1/255,rotation_range=30,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.5,zoom_range=0.3,horizontal_flip=True,vertical_flip=True,validation_split=0.2)

# 저장된 데이터 불러오기
# X_train, X_test, y_train, y_test,X_train_f, X_vailed, y_train_f, y_vailed = np.load('./HandGesture/multi_image_data.npy',allow_pickle=True)

xy = np.load('./HandGesture/multi_image_data.npz')
X_train = xy['X_train']
X_test = xy['X_test']
y_train = xy['y_train']
y_test = xy['y_test']
X_train_f = xy['X_train_f']
X_vailed = xy['X_vailed']
y_train_f = xy['y_train_f']
y_vailed = xy['y_vailed']


#학습집합의 데이터 증식
train_generator = train_datagen.flow(
    x=X_train,y=y_train,
    batch_size = 32,
    shuffle = True
)

#최종 학습 집합 데이터 증식
train_f_generator = train_datagen.flow(
    x=X_train_f,y=y_train_f,
    batch_size = 32,
    shuffle = True
)

#검증 집합 데이터 증식
vailed_generator = train_datagen.flow(
    x=X_vailed,y=y_vailed,
    batch_size = 32,
    shuffle = True
)

#테스트 집합 데이터 증식
test_generator = train_datagen.flow(
    x=X_test,y=y_test,
    batch_size = 32,
    shuffle = True
)

In [5]:
#카테고리 설정
categories = ["O", "V", "paper", "rock","side"] 
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255
X_train_f = X_train_f.astype(float) /255
X_vailed = X_vailed.astype(float) /255

# 그냥 인터넷에서 가져온 신경망 구조

In [6]:
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', kernel_initializer='he_uniform', padding='same')

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[64, 64, 3]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(), # 2차원을 1차원으로 변환하여 밀집층으로 연결
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(nb_classes, activation='softmax'),
])
model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

model_dir = './model'

checkpoint_cb = keras.callbacks.ModelCheckpoint("model1.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history_1 = model.fit_generator(train_f_generator, epochs=10, validation_data=vailed_generator,
                   callbacks=[checkpoint_cb, early_stopping_cb])
model.save("./model/model_1")

Epoch 1/10


C:\Users\sungm\AppData\Local\Temp\ipykernel_12312\4009080640.py:27: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_1 = model.fit_generator(train_f_generator, epochs=10, validation_data=vailed_generator,


40/40 [==============================] - 19s 408ms/step - loss: 1.7178 - accuracy: 0.2414 - val_loss: 1.5878 - val_accuracy: 0.3429
Epoch 2/10


C:\Users\sungm\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


40/40 [==============================] - 15s 374ms/step - loss: 1.5520 - accuracy: 0.2958 - val_loss: 1.4531 - val_accuracy: 0.3981
Epoch 3/10
40/40 [==============================] - 15s 370ms/step - loss: 1.4662 - accuracy: 0.3573 - val_loss: 1.2554 - val_accuracy: 0.5396
Epoch 4/10
40/40 [==============================] - 15s 376ms/step - loss: 1.3446 - accuracy: 0.4420 - val_loss: 1.1699 - val_accuracy: 0.5492
Epoch 5/10
40/40 [==============================] - 16s 388ms/step - loss: 1.2687 - accuracy: 0.4724 - val_loss: 1.1919 - val_accuracy: 0.5923
Epoch 6/10
40/40 [==============================] - 15s 381ms/step - loss: 1.1690 - accuracy: 0.5516 - val_loss: 1.1714 - val_accuracy: 0.5875
Epoch 7/10
40/40 [==============================] - 15s 376ms/step - loss: 1.0605 - accuracy: 0.5963 - val_loss: 0.8239 - val_accuracy: 0.6835
Epoch 8/10
40/40 [==============================] - 15s 377ms/step - loss: 0.8715 - accuracy: 0.6627 - val_loss: 0.6313 - val_accuracy: 0.7722
Epoch 9/10

INFO:tensorflow:Assets written to: ./model/model_1\assets


# 합성곱 + 최대풀링 섞은 신경망

In [7]:
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', kernel_initializer='he_uniform', padding='same')

model2 = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[64, 64, 3]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(nb_classes, activation='softmax'),
])

model2.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

model_dir = './model'

checkpoint_cb = keras.callbacks.ModelCheckpoint("model2.h5", save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history_2 = model2.fit_generator(train_f_generator, epochs=10, validation_data=vailed_generator,
                   callbacks=[checkpoint_cb, early_stopping_cb])
model2.save("./model/model_2")

Epoch 1/10


C:\Users\sungm\AppData\Local\Temp\ipykernel_12312\1441680498.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_2 = model2.fit_generator(train_f_generator, epochs=10, validation_data=vailed_generator,


40/40 [==============================] - 8s 172ms/step - loss: 1.4720 - accuracy: 0.4468 - val_loss: 1.1034 - val_accuracy: 0.6043
Epoch 2/10


C:\Users\sungm\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


40/40 [==============================] - 7s 174ms/step - loss: 1.0462 - accuracy: 0.6035 - val_loss: 1.0371 - val_accuracy: 0.6043
Epoch 3/10
40/40 [==============================] - 7s 174ms/step - loss: 0.8901 - accuracy: 0.6579 - val_loss: 0.7148 - val_accuracy: 0.7770
Epoch 4/10
40/40 [==============================] - 7s 165ms/step - loss: 0.7820 - accuracy: 0.7570 - val_loss: 0.7066 - val_accuracy: 0.7818
Epoch 5/10
40/40 [==============================] - 6s 159ms/step - loss: 0.6331 - accuracy: 0.7794 - val_loss: 0.5836 - val_accuracy: 0.8010
Epoch 6/10
40/40 [==============================] - 6s 160ms/step - loss: 0.4911 - accuracy: 0.8345 - val_loss: 0.4525 - val_accuracy: 0.8657
Epoch 7/10
40/40 [==============================] - 6s 164ms/step - loss: 0.4344 - accuracy: 0.8609 - val_loss: 0.4425 - val_accuracy: 0.8657
Epoch 8/10
40/40 [==============================] - 6s 160ms/step - loss: 0.4818 - accuracy: 0.8441 - val_loss: 0.5084 - val_accuracy: 0.8345
Epoch 9/10
40/40 

INFO:tensorflow:Assets written to: ./model/model_2\assets


# VGG 모델 개선

In [8]:
def model_maker():
    base_model = VGG16(include_top=False, input_shape=(64, 64, 3)) #베이스 모델로는 VGG16 모델을

    for layer in base_model.layers[:-2]:
        layer.trainable = False # Top 층을 제외한 나머지 층에서 2개의 층을 새롭게 학습 
 
    input1 = Input(shape=(64, 64, 3))
    custom_model = base_model(input1)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(nb_classes, activation='softmax')(custom_model)
    return Model(inputs=input1, outputs=predictions)
model3 = model_maker()

model3.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])

model_dir = './model'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = 'model3.h5'
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

history_3 = model3.fit_generator(train_f_generator, epochs=10, validation_data=vailed_generator,
                   callbacks=[checkpoint_cb, early_stopping_cb])

model3.save("./model/model_3")

Epoch 1/10


C:\Users\sungm\AppData\Local\Temp\ipykernel_12312\179852561.py:29: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_3 = model3.fit_generator(train_f_generator, epochs=10, validation_data=vailed_generator,


40/40 [==============================] - 13s 300ms/step - loss: 1.0228 - acc: 0.5939 - val_loss: 0.4512 - val_acc: 0.8561
Epoch 2/10
40/40 [==============================] - 12s 295ms/step - loss: 0.5435 - acc: 0.8129 - val_loss: 0.3015 - val_acc: 0.8825
Epoch 3/10


C:\Users\sungm\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


40/40 [==============================] - 12s 301ms/step - loss: 0.4437 - acc: 0.8385 - val_loss: 0.2956 - val_acc: 0.8753
Epoch 4/10
40/40 [==============================] - 12s 300ms/step - loss: 0.3822 - acc: 0.8721 - val_loss: 0.2244 - val_acc: 0.9161
Epoch 5/10
40/40 [==============================] - 12s 299ms/step - loss: 0.3647 - acc: 0.8793 - val_loss: 0.2065 - val_acc: 0.9257
Epoch 6/10
40/40 [==============================] - 12s 298ms/step - loss: 0.3151 - acc: 0.8841 - val_loss: 0.1566 - val_acc: 0.9424
Epoch 7/10
40/40 [==============================] - 12s 296ms/step - loss: 0.2838 - acc: 0.9017 - val_loss: 0.1639 - val_acc: 0.9424
Epoch 8/10
40/40 [==============================] - 12s 296ms/step - loss: 0.2722 - acc: 0.9089 - val_loss: 0.1987 - val_acc: 0.9400
Epoch 9/10
40/40 [==============================] - 12s 295ms/step - loss: 0.2182 - acc: 0.9265 - val_loss: 0.2046 - val_acc: 0.9281
Epoch 10/10
40/40 [==============================] - 12s 308ms/step - loss: 0.22

INFO:tensorflow:Assets written to: ./model/model_3\assets


In [10]:
print("모델1 정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))
print("모델2(합성곱+최대풀링) 정확도 : %.4f" % (model2.evaluate(X_test, y_test)[1]))
print("모델3(VGG모델) 정확도 : %.4f" % (model3.evaluate(X_test, y_test)[1]))

18/18 [==============================] - 1s 74ms/step - loss: 0.4778 - accuracy: 0.8417
모델1 정확도 : 0.8417
18/18 [==============================] - 1s 36ms/step - loss: 0.2582 - accuracy: 0.9245
모델2(합성곱+최대풀링) 정확도 : 0.9245
18/18 [==============================] - 3s 178ms/step - loss: 0.1473 - acc: 0.9388
모델3(VGG모델) 정확도 : 0.9388


# vgg로 최종 모델 학습

In [12]:
model4= model3

In [13]:
#최종모델 불러오기
# model4 = load_model('./model/model_Last')

history_4 = model4.fit_generator(train_generator,
                        epochs=100,
                        validation_data=test_generator,
                        callbacks=[checkpoint_cb, early_stopping_cb])
model4.save("./model/model_Last")


Epoch 1/100


C:\Users\sungm\AppData\Local\Temp\ipykernel_12312\2840650790.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_4 = model4.fit_generator(train_generator,


53/53 [==============================] - 15s 287ms/step - loss: 0.2325 - acc: 0.9221 - val_loss: 0.1611 - val_acc: 0.9335
Epoch 2/100
53/53 [==============================] - 15s 286ms/step - loss: 0.1849 - acc: 0.9269 - val_loss: 0.1579 - val_acc: 0.9406
Epoch 3/100
53/53 [==============================] - 15s 286ms/step - loss: 0.1938 - acc: 0.9353 - val_loss: 0.1691 - val_acc: 0.9442
Epoch 4/100
53/53 [==============================] - 15s 287ms/step - loss: 0.1782 - acc: 0.9382 - val_loss: 0.1530 - val_acc: 0.9496
Epoch 5/100


C:\Users\sungm\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


53/53 [==============================] - 15s 292ms/step - loss: 0.1805 - acc: 0.9347 - val_loss: 0.1621 - val_acc: 0.9388
Epoch 6/100
53/53 [==============================] - 15s 293ms/step - loss: 0.1535 - acc: 0.9454 - val_loss: 0.1373 - val_acc: 0.9478
Epoch 7/100
53/53 [==============================] - 15s 286ms/step - loss: 0.1580 - acc: 0.9460 - val_loss: 0.1558 - val_acc: 0.9424
Epoch 8/100
53/53 [==============================] - 15s 290ms/step - loss: 0.1735 - acc: 0.9430 - val_loss: 0.1566 - val_acc: 0.9478
Epoch 9/100
53/53 [==============================] - 15s 285ms/step - loss: 0.1306 - acc: 0.9502 - val_loss: 0.1617 - val_acc: 0.9478
Epoch 10/100
53/53 [==============================] - 15s 285ms/step - loss: 0.1475 - acc: 0.9562 - val_loss: 0.1574 - val_acc: 0.9478
Epoch 11/100
53/53 [==============================] - 15s 285ms/step - loss: 0.1339 - acc: 0.9514 - val_loss: 0.1471 - val_acc: 0.9478
Epoch 12/100
53/53 [==============================] - 16s 295ms/step - l

INFO:tensorflow:Assets written to: ./model/model_Last\assets


## 최종 모델 성능 (테스트 집합)

In [15]:
print("최종 모델 정확도 : %.4f" % (model4.evaluate(X_test, y_test)[1]))

18/18 [==============================] - 3s 178ms/step - loss: 0.0678 - acc: 0.9766
최종 모델 정확도 : 0.9766


# 일반 사진으로 테스트

In [16]:
caltech_dir = "./HandGesture/test"
image_w = 64
image_h = 64

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")

for i, f in enumerate(files):
    img = cv2.imread(f)#이미지 열고
    #손 색상 변경
    ycrb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
    img = cv2.inRange(ycrb,np.array([0,133,77]),np.array([255,173,127])) 
    img = Image.fromarray(img)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    #img.show()
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
model = load_model('./model/model_Last')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0

#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    print(i)
    print(pre_ans)
    pre_ans_str = ''
    if pre_ans == 0: pre_ans_str = "o"
    elif pre_ans == 1: pre_ans_str = "V"
    elif pre_ans == 2: pre_ans_str = "paper"
    elif pre_ans == 3: pre_ans_str = "rock"
    else: pre_ans_str = "side"
   
    if i[0] >= 0.8 : print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[1] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
    if i[2] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[3] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[4] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    cnt += 1

1/1 [==============================] - 0s 298ms/step
[0.000 0.000 1.000 0.000 0.000]
2
해당 20221127_213403_001.jpg이미지는 paper로 추정됩니다.
[0.000 0.000 1.000 0.000 0.000]
2
해당 20221127_213403_002.jpg이미지는 paper로 추정됩니다.
[0.000 0.000 1.000 0.000 0.000]
2
해당 20221127_213403_003.jpg이미지는 paper로 추정됩니다.
[0.000 0.000 1.000 0.000 0.000]
2
해당 20221127_213403_004.jpg이미지는 paper로 추정됩니다.
[0.000 0.000 1.000 0.000 0.000]
2
해당 20221127_213403_005.jpg이미지는 paper로 추정됩니다.
[0.000 1.000 0.000 0.000 0.000]
1
해당 20221127_214509_001.jpg이미지는 V으로 추정됩니다.
[0.000 1.000 0.000 0.000 0.000]
1
해당 20221127_214509_002.jpg이미지는 V으로 추정됩니다.
[0.000 1.000 0.000 0.000 0.000]
1
해당 20221127_214509_003.jpg이미지는 V으로 추정됩니다.
[0.000 1.000 0.000 0.000 0.000]
1
해당 20221127_214509_004.jpg이미지는 V으로 추정됩니다.
[0.000 1.000 0.000 0.000 0.000]
1
해당 20221127_214509_005.jpg이미지는 V으로 추정됩니다.
[0.000 0.000 1.000 0.000 0.000]
2
해당 20221127_214801_084.jpg이미지는 paper로 추정됩니다.
[0.000 0.000 1.000 0.000 0.000]
2
해당 20221127_214812_040.jpg이미지는 paper로 추정됩니다.
[0.000 0.000 1

# 실제 화면으로 테스트

In [18]:
from tkinter import * #GUI
import numpy as np 
import cv2 #웹캠
import screen_brightness_control as sbc #화면 밝기 조절
import pyautogui #볼륨 업 다운
import keyboard
from PIL import Image
import os, glob, numpy as np
from keras.models import load_model

def chgkey(x):
    if x == 1: #o
        pyautogui.press('volumeup')
    elif x == 2: #v
        pyautogui.press('volumedown')
    elif x == 3: #r
        pyautogui.press('volumemute')   

def cam():
    cv2.imwrite("self camera test.jpg", frame) # 사진 저장


brightness = sbc.get_brightness()
b = brightness[0]

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
y=1

while(True):
    ret, frame = capture.read() 
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2YCrCb)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cam()
        
        caltech_dir = "./"
        image_w = 64
        image_h = 64

        X = []
        filenames = []
        files = glob.glob(caltech_dir+"self camera test.jpg")

        for i, f in enumerate(files):
            img = cv2.imread(f)#이미지 열고
            #손 색상 변경
            ycrb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
            img = cv2.inRange(ycrb,np.array([0,133,77]),np.array([255,173,127])) 
            img = Image.fromarray(img)
            img = img.convert("RGB")
            img = img.resize((image_w, image_h))
            #img.show()
            data = np.asarray(img)
            filenames.append(f)
            X.append(data)

        X = np.array(X)
        model = load_model('./model/model_Last')

        prediction = model.predict(X)
        np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
        cnt = 0
        #이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
        for i in prediction:
            pre_ans = i.argmax()  # 예측 레이블
            print(i)
            print(pre_ans)
            pre_ans_str = ''
            if pre_ans == 0: pre_ans_str = "o"
            elif pre_ans == 1: pre_ans_str = "V"
            elif pre_ans == 2: pre_ans_str = "paper"
            elif pre_ans == 3: pre_ans_str = "rock"
            elif pre_ans == 4: pre_ans_str = "side"
            
            if i[0] >= 0.8 : 
                print("o")
                y=4
            if i[1] >= 0.8: 
                print("v")
                y=5
            if i[2] >= 0.8:
                print("paper") 
                y=1
            if i[3] >= 0.8:
                print("rock") 
                y=2
            if i[4] >= 0.8:
                print("side") 
                y=3
            cnt += 1

        if y == 1:
            chgkey(1)
        elif y == 2:
            chgkey(2)
        elif y == 3:
            chgkey(3)
        elif y == 4:
            brightness[0] -= 10
            if brightness[0] < 0 :
                brightness[0] = 0
            sbc.set_brightness(brightness[0])
        elif y == 5:
            brightness[0] += 10
            if brightness[0] > 100:
                brightness[0] = 100
            sbc.set_brightness(brightness[0])
        else : 
            print("인식되지 않음")
    elif 0xFF == ord('w'):
        capture.release()
        cv2.destroyAllWindows()
        break  

1/1 [==============================] - 0s 127ms/step
[0.000 0.000 1.000 0.000 0.000]
2
paper
1/1 [==============================] - 0s 119ms/step
[0.000 0.000 1.000 0.000 0.000]
2
paper


KeyboardInterrupt: 